In [1]:
import sys
sys.path.append(r'C:\Users\Mi\Desktop\code\git\python-utils-ak')
from utils_ak.interactive_imports import *

def _import_globals(module):
    for x in dir(module):
        if not x.startswith('_'):
            globals()[x] = getattr(module, x)
            
def _load(module_obj, module_name=None, reload=False, import_globals=False):
    module = load(module_obj, module_name, reload, import_globals=False)
    if import_globals:
        _import_globals(module)
    return module

def load_sources():
    _load('src/time.py', import_globals=True, reload=True)
    _load('src/block.py', import_globals=True, reload=True)
    _load('src/color.py', import_globals=True, reload=True)
    _load('src/drawing.py', import_globals=True, reload=True)
    _load('src/interval.py', import_globals=True, reload=True)

In [2]:
load_sources()

# Models

In [3]:
import sys
# sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/2020.10-umalat/umalat')
sys.path.append(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat')
from app.models import *
from app.models import Boiling as BoilingModel
from app.models import Termizator as TermizatorModel

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# sqlite_filepath = '/home/akadaner/Desktop/data.sqlite'
sqlite_filepath = r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite"
engine = create_engine(f"sqlite:///{sqlite_filepath}")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

def cast_sku(obj):
    if isinstance(obj, SKU):
        return obj
    elif isinstance(obj, (str, int)):
        obj = str(obj)
        return session.query(SKU).filter(SKU.id == obj).first()
    else:
        raise Exception('Unknown sku type')
        
def cast_boiling(obj):
    if isinstance(obj, Boiling):
        return obj
    elif isinstance(obj, (str, int)):
        obj = str(obj)
        return session.query(Boiling).filter(Boiling.id == obj).first()
    else:
        raise Exception('Unknown boiling type')

In [4]:
values = session.query(SKU).all()
skus = pd.DataFrame(values, columns=['sku'])
skus['name'] = skus['sku'].apply(lambda sku:sku.name)
skus['percent'] = skus['sku'].apply(lambda sku:sku.boiling.percent)
skus.head()

,sku,name,percent
0,<SKU 1>,"Моцарелла ""Pretto"" (для бутербродов), 45%, 0,2 кг, т/ф, (9 шт)",2.7
1,<SKU 2>,"Моцарелла ""Pretto"", 45%, 1,2 кг, в/у",2.7
2,<SKU 3>,"Моцарелла ""Unagrande"", 45%, 0,12 кг, ф/п (кубики)",2.7
3,<SKU 4>,"Моцарелла ""Unagrande"", 45%, 3 кг, пл/л",2.7
4,<SKU 5>,"Моцарелла (палочки), 45%, кг, пл/л",2.7


In [33]:
values = session.query(Boiling).all()
boilings = pd.DataFrame(values, columns=['boiling'])
boilings['percent'] = boilings['boiling'].apply(lambda b: b.percent)
boilings

,boiling,percent
0,<Boiling 1>,2.7
1,<Boiling 2>,2.7
2,<Boiling 3>,2.7
3,<Boiling 4>,2.7
4,<Boiling 5>,3.3
5,<Boiling 6>,3.3
6,<Boiling 7>,3.3
7,<Boiling 8>,3.3
8,<Boiling 9>,3.6
9,<Boiling 10>,3.6


In [38]:
boilings.iloc[0]['boiling'].percent, boilings.iloc[0]['boiling'].

2.7

In [6]:
TERMIZATOR = session.query(Termizator).first()
TERMIZATOR.pouring_time = 30

In [7]:
def get_sample_boiling_conf(boiling_type='water'):
    if boiling_type == 'water':
        tmp = skus[skus['percent'] == 3.3]
        sku = tmp.iloc[0]['sku']
        BOILING_CONF = sku.boiling
        BOILING_REQUEST = {sku: 850}
        return BOILING_CONF, BOILING_REQUEST
    elif boiling_type == 'salt':
        tmp = skus[skus['percent'] == 2.7]
        sku = tmp.iloc[0]['sku']
        BOILING_CONF = sku.boiling
        BOILING_REQUEST = {sku: 850}
        return BOILING_CONF, BOILING_REQUEST

In [39]:
def make_melting_and_packing(boiling_conf, boiling_request, boiling_type, 
                            melting_line=None):
    maker = BlockMaker(default_push_func=dummy_push)
    make = maker.make
    
    with make('melting_and_packing'):
        # packing and melting time
        
        if boiling_type == 'water':
            melting_time = custom_round(850 / boiling_conf.meltings.speed * 60 * 1.3, 5, rounding='ceil')
            packing_time = 0

            # todo: make proper order
            for sku, sku_kg in boiling_request.items():
                packing_speed = min(sku.packing_speed, boiling_conf.meltings.speed)
                packing_time += custom_round(sku_kg / packing_speed * 60, 5, rounding='ceil')
            
            full_melting_time = boiling_conf.meltings.serving_time + melting_time
            
        elif boiling_type == 'salt':
            packing_time = 0

            # todo: make proper order
            for sku, sku_kg in boiling_request.items():
                packing_time += custom_round(sku_kg / sku.packing_speed * 60, 5, rounding='ceil')

            melting_time = packing_time
            
            full_melting_time = boiling_conf.meltings.serving_time + melting_time + boiling_conf.meltings.salting_time
        
        # todo: make proper order
        form_factor_label = ' / '.join([str(sku.form_factor) for sku in boiling_request.keys()])
        brand_label = ' / '.join([sku.brand_name for sku in boiling_request.keys()])
        
        with make('melting', y=10, time_size=full_melting_time, melting_line=melting_line):
            with make(y=0):
                make('melting_label', time_size=4 * 5)
                make('melting_name', time_size=full_melting_time - 4 * 5, form_factor_label=form_factor_label)
            with make(y=1):
                make('serving', time_size=boiling_conf.meltings.serving_time)
                make('melting_process', time_size=melting_time, speed=boiling_conf.meltings.speed)

            with make(y=2):
                make(time_size=boiling_conf.meltings.serving_time, visible=False)
                if boiling_type == 'water':
                    make('cooling1', time_size=boiling_conf.meltings.first_cooling_time)
                    make('cooling2', time_size=boiling_conf.meltings.second_cooling_time)
                elif boiling_type == 'salt':
                    make('salting', time_size=boiling_conf.meltings.salting_time)

        # prepare time
        if boiling_type == 'water':
            prepare_time = boiling_conf.meltings.serving_time + boiling_conf.meltings.first_cooling_time + boiling_conf.meltings.second_cooling_time
        elif boiling_type == 'salt':
            prepare_time = boiling_conf.meltings.serving_time + boiling_conf.meltings.salting_time

        with make('packing', t=prepare_time / 5, time_size=packing_time, push_func=add_push):
            with make(y=0):
                make('packing_label', time_size=3 * 5)
                make('packing_name', time_size=packing_time - 3 * 5, form_factor_label=form_factor_label)
            with make(y=1):
                make('packing_brand', time_size=packing_time, brand_label=brand_label)

                    # todo: add configurations
        #         with make(y=2):
        #             make('configuration', size=3, visible=False)
        #             make('configuration', size=1)
        #             make('configuration', size=5, visible=False)
        #             make('configuration', size=1)

    res = maker.root.children[0]
    res.rel_props['size'] = max(c.end for c in res.children)
    
    return res

BOILING_CONF, BOILING_REQUEST = get_sample_boiling_conf('salt')
block = make_melting_and_packing(BOILING_CONF, BOILING_REQUEST, 'salt', melting_line='0')
print(draw_print(block))

========================================= melting_and_packing (0,41]
========================================= melting (0,41]
==== melting_label (0,4]
    ===================================== melting_name (4,41]
====== serving (0,6]
      ========== melting_process (6,16]
====== block (0,6]
      ========================= salting (6,31]
                               ========== packing (31.0,41.0]
                               === packing_label (0,3]
                                  ======= packing_name (3,10]
                               ========== packing_brand (0,10]



In [41]:
def make_boiling(boiling_conf, boiling_request, boiling_type='water', block_num=12, pouring_line=None, melting_line=None):
    maker = BlockMaker(default_push_func=dummy_push)
    make = maker.make
    
    boiling_label = '{} {} {} 8000кг'.format(boiling_conf.percent, boiling_conf.ferment, 'с лактозой' if boiling_conf.is_lactose else 'безлактозная')


    with make('boiling', block_num=block_num, boiling_type=boiling_type, boiling_label=boiling_label):
        # todo: make boiling size

        timings = []
        timings.append(boiling_conf.pourings.pouring_time)
        timings.append(boiling_conf.pourings.soldification_time)
        timings.append(boiling_conf.pourings.cutting_time)
        timings.append(boiling_conf.pourings.pouring_off_time)
        timings.append(boiling_conf.pourings.extra_time)
        
        with make('pouring', time_size=sum(timings), pouring_line=pouring_line):
            with make(y=0):
                make('termizator', time_size=TERMIZATOR.pouring_time)
                make('pouring_name', time_size=sum(timings) - TERMIZATOR.pouring_time)
            with make(y=1):
                make('pouring_and_fermenting', time_size=timings[0])
                make('soldification', time_size=timings[1])
                make('cutting', time_size=timings[2])
                make('pouring_off', time_size=timings[3])
                make('extra', time_size=timings[4])

        # todo: specify parameter
        make('drenator', size=cast_t('04:00'), visible=False)
        
        make(make_melting_and_packing(boiling_conf, boiling_request, boiling_type, melting_line=melting_line))
        

    res = maker.root.children[0]
    res.rel_props['size'] = max(c.end for c in res.children)

    return res

BOILING_CONF, BOILING_REQUEST = get_sample_boiling_conf('salt')
block = make_boiling(BOILING_CONF, BOILING_REQUEST, 'salt', pouring_line='0', melting_line='0')
print(draw_print(block))

==================================================================================================================== boiling (0,116]
=========================== pouring (0,27]
====== termizator (0,6]
      ===================== pouring_name (6,27]
============ pouring_and_fermenting (0,12]
            ====== soldification (12,18]
                  ==== cutting (18,22]
                      === pouring_off (22,25]
                         == extra (25,27]
                           ================================================ drenator (27,75]
                                                                           ========================================= melting_and_packing (75,116]
                                                                           ========================================= melting (0,41]
                                                                           ==== melting_label (0,4]
                                                                      

# Drawing

In [42]:
BASE_STYLE = {
    'pouring': {'beg_time': '01:00'},
    'termizator': {'text': '{block_num} налив'},
    'pouring_name': {'text': '{boiling_label}'},
    'pouring_and_fermenting': {'text': 'налив/внесение закваски', 'color': 'orange'},
    'soldification': {'text': 'схватка', 'color': 'yellow'},
    'cutting': {'text': 'резка/обсушка', 'color': '#92d050'},
    'pouring_off': {'text': 'слив', 'color': 'red'},
    'melting': {'beg_time': '07:00'},
    'melting_label': {'text': '{block_num}'},
    'melting_name': {'text': '{form_factor_label}'},
    'serving': {'color': 'orange', 'text': 'подача и вымешивание'},
    'melting_process': {'color': 'orange', 'text': lambda b: 'плавление/формирование {speed} кг/ч' if b.abs_props['boiling_type'] == 'water' else 'плавление/формирование'},
    'cooling1': {'text': 'охлаж1 <{size} * 5> мин'},
    'cooling2': {'text': 'охлаж2 <{size} * 5> мин'},
    'packing': {'beg_time': '07:00'},
    'packing_label': {'text': '{block_num}'},
    'packing_name': {'text': '{form_factor_label}'},
    'packing_brand': {'color': '#f2dcdb', 'text': '{brand_label}'},
    'configuration': {'color': 'red'},
    'salting': {'color': 'green', 'text': 'посолка'}
}


ROW_STYLE_DEV = {
    'pouring': {'y': lambda b: {'0': 1, '1': 3}[b.abs_props['pouring_line']] if b.abs_props['boiling_type'] == 'water' else {'0': 5, '1': 7}[b.abs_props['pouring_line']]},
    'melting': {'y': lambda b: 9 if b.abs_props['boiling_type'] == 'water' else {'0': 14, '1': 17, '2': 20, '3': 23}[b.abs_props['melting_line']]},
    'packing': {'y': lambda b: 12 if b.abs_props['boiling_type'] == 'water' else 26}

}

ROW_STYLE_PROD = {
    'pouring': {'y': lambda b: {'0': 6, '1': 9}[b.abs_props['pouring_line']] if b.abs_props['boiling_type'] == 'water' else {'0': 15, '1': 18}[b.abs_props['pouring_line']]},
    'melting': {'y': lambda b: 24 if b.abs_props['boiling_type'] == 'water' else {'0': 33, '1': 36, '2': 39, '3': 42}[b.abs_props['melting_line']]},
    'packing': {'y': lambda b: 29 if b.abs_props['boiling_type'] == 'water' else 46}

}

ROW_STYLE = ROW_STYLE_DEV

styles = [BASE_STYLE, ROW_STYLE]

STYLE = {}
for style in styles:
    for k, v in style.items():
        if k not in STYLE:
            STYLE[k] = v
        else:
            STYLE[k].update(v)
STYLE

{'pouring': {'beg_time': '01:00', 'y': <function __main__.<lambda>(b)>},
 'termizator': {'text': '{block_num} налив'},
 'pouring_name': {'text': '{boiling_label}'},
 'pouring_and_fermenting': {'text': 'налив/внесение закваски',
  'color': 'orange'},
 'soldification': {'text': 'схватка', 'color': 'yellow'},
 'cutting': {'text': 'резка/обсушка', 'color': '#92d050'},
 'pouring_off': {'text': 'слив', 'color': 'red'},
 'melting': {'beg_time': '07:00', 'y': <function __main__.<lambda>(b)>},
 'melting_label': {'text': '{block_num}'},
 'melting_name': {'text': '{form_factor_label}'},
 'serving': {'color': 'orange', 'text': 'подача и вымешивание'},
 'melting_process': {'color': 'orange',
  'text': <function __main__.<lambda>(b)>},
 'cooling1': {'text': 'охлаж1 <{size} * 5> мин'},
 'cooling2': {'text': 'охлаж2 <{size} * 5> мин'},
 'packing': {'beg_time': '07:00', 'y': <function __main__.<lambda>(b)>},
 'packing_label': {'text': '{block_num}'},
 'packing_name': {'text': '{form_factor_label}'},
 '

In [43]:
load_sources()

In [44]:
root = Block('root', t=cast_t('01:45'))
BOILING_CONF, BOILING_REQUEST = get_sample_boiling_conf('salt')
block = make_boiling(BOILING_CONF, BOILING_REQUEST, 'salt', melting_line='0', pouring_line='1')
dummy_push(root, block)
draw_schedule(root, STYLE, 'output.xlsx', init_sheet)

termizator 7 (14,20] (21,27]
pouring_name 7 (20,41] (27,48]
pouring_and_fermenting 8 (14,26] (21,33]
soldification 8 (26,32] (33,39]
cutting 8 (32,36] (39,43]
pouring_off 8 (36,39] (43,46]
extra 8 (39,41] (46,48]
melting_label 14 (17,21] (96,100]
melting_name 14 (21,58] (100,137]
serving 15 (17,23] (96,102]
melting_process 15 (23,33] (102,112]
salting 16 (23,48] (102,127]
packing_label 26 (48.0,51.0] (127.0,130.0]
packing_name 26 (51.0,58.0] (130.0,137.0]
packing_brand 27 (48.0,58.0] (127.0,137.0]


PermissionError: [Errno 13] Permission denied: 'output.xlsx'

# Schedule

In [15]:
def boiling_validator(parent, boiling):
    if not parent.children:
        return
    # todo: optimize
    #     b1 = max(parent.children, key=lambda b: b.rel_props.get('t', 0))

    for b1 in parent.children[-4:]:
        b2 = boiling

        b1.rel_props['props_mode'] = 'absolute'
        b2.rel_props['props_mode'] = 'absolute'

        b1.upd_abs_props()
        b2.upd_abs_props()

        validate_disjoint(b1['pouring'][0]['termizator'], b2['pouring'][0]['termizator'])

        if b1.props['boiling_type'] == b2.props['boiling_type']:
            boiling_type = b1.props['boiling_type']

            if b1['pouring'].props['pouring_line'] == b2['pouring'].props['pouring_line']:
                validate_disjoint(b1['pouring'], b2['pouring'])

            if boiling_type == 'water':
                validate_disjoint(b1['melting_and_packing']['melting'], b2['melting_and_packing']['melting'])
            else:
                if b1['melting_and_packing']['melting'].props['melting_line'] == b2['melting_and_packing']['melting'].props['melting_line']:
                    validate_disjoint(b1['melting_and_packing']['melting'], b2['melting_and_packing']['melting'])
                else:
                    validate_disjoint(b1['melting_and_packing']['melting'][1]['melting_process'], b2['melting_and_packing']['melting'][1]['melting_process'])

            validate_disjoint(b1['melting_and_packing']['packing'], b2['melting_and_packing']['packing'])

        b1.rel_props.pop('props_mode')
        b2.rel_props.pop('props_mode')

        b1.upd_abs_props()
        b2.upd_abs_props()

In [16]:
from itertools import product

iter_water_props = [{'pouring_line': str(v)} for v in [0, 1]]
iter_salt_props = [{'pouring_line': str(v[0]), 'melting_line': str(v[1])} for v in product([0, 1], [0, 1, 2, 3])]
iter_water_props, iter_salt_props

([{'pouring_line': '0'}, {'pouring_line': '1'}],
 [{'pouring_line': '0', 'melting_line': '0'},
  {'pouring_line': '0', 'melting_line': '1'},
  {'pouring_line': '0', 'melting_line': '2'},
  {'pouring_line': '0', 'melting_line': '3'},
  {'pouring_line': '1', 'melting_line': '0'},
  {'pouring_line': '1', 'melting_line': '1'},
  {'pouring_line': '1', 'melting_line': '2'},
  {'pouring_line': '1', 'melting_line': '3'}])

In [17]:
load_sources()

In [19]:
# draw salt and water
root = Block('root', t=cast_t('01:45'))

boilings = []

def gen_boiling(boiling_type):
    BOILING_CONF, BOILING_REQUEST = get_sample_boiling_conf(boiling_type)
    return make_boiling(BOILING_CONF, BOILING_REQUEST, boiling_type)

for i in range(2):
    boilings.append(gen_boiling('water'))
    
for i in range(2):
    boilings.append(gen_boiling('salt'))

for b in boilings:
    iter_props = iter_water_props if b.props['boiling_type'] == 'water' else iter_salt_props
    dummy_push(root, b, iter_props=iter_props, validator=boiling_validator, beg='last_beg', max_tries=100)

draw_schedule(root, STYLE, 'output.xlsx', init_sheet)
root

termizator 6 (14,20] (21,27]
pouring_name 6 (20,40] (27,47]
pouring_and_fermenting 7 (14,22] (21,29]
soldification 7 (22,28] (29,35]
cutting 7 (28,35] (35,42]
pouring_off 7 (35,38] (42,45]
extra 7 (38,40] (45,47]
melting_label 24 (16,20] (95,99]
melting_name 24 (20,37] (99,116]
serving 25 (16,22] (95,101]
melting_process 25 (22,37] (101,116]
cooling1 26 (22,27] (101,106]
cooling2 26 (27,37] (106,116]
packing_label 29 (37.0,40.0] (116.0,119.0]
packing_name 29 (40.0,49.0] (119.0,128.0]
packing_brand 30 (37.0,49.0] (116.0,128.0]
termizator 9 (35,41] (42,48]
pouring_name 9 (41,61] (48,68]
pouring_and_fermenting 10 (35,43] (42,50]
soldification 10 (43,49] (50,56]
cutting 10 (49,56] (56,63]
pouring_off 10 (56,59] (63,66]
extra 10 (59,61] (66,68]
melting_label 24 (37,41] (116,120]
melting_name 24 (41,58] (120,137]
serving 25 (37,43] (116,122]
melting_process 25 (43,58] (122,137]
cooling1 26 (43,48] (122,127]
cooling2 26 (48,58] (127,137]
packing_label 29 (58.0,61.0] (137.0,140.0]
packing_name

root (21, 21]
  boiling (0, 107]
    pouring (0, 26]
      block (0, 0]
        termizator (0, 6]
        pouring_name (6, 26]
      block (0, 0]
        pouring_and_fermenting (0, 8]
        soldification (8, 14]
        cutting (14, 21]
        pouring_off (21, 24]
        extra (24, 26]
    drenator (26, 74]
    melting_and_packing (74, 107]
      melting (0, 21]
        block (0, 0]
          melting_label (0, 4]
          melting_name (4, 21]
        block (0, 0]
          serving (0, 6]
          melting_process (6, 21]
        block (0, 0]
          block (0, 6]
          cooling1 (6, 11]
          cooling2 (11, 21]
      packing (21.0, 33.0]
        block (0, 0]
          packing_label (0, 3]
          packing_name (3, 12]
        block (0, 0]
          packing_brand (0, 12]
  boiling (21, 128]
    pouring (0, 26]
      block (0, 0]
        termizator (0, 6]
        pouring_name (6, 26]
      block (0, 0]
        pouring_and_fermenting (0, 8]
        soldification (8, 14]
     

# Final Water Algo

In [20]:
import anyconfig
request = anyconfig.load('2020.11.22 sample_day_request.yaml')
date = cast_datetime(request['Date'])

values = []
for boiling in request['Boilings']:
    for sku_id, sku_kg in boiling['SKUVolumes'].items():
        values.append([boiling['BoilingId'], sku_id, sku_kg])
df = pd.DataFrame(values)

values = []
for boiling_id, boiling_grp in df.groupby(0):
    boiling_dic = boiling_grp[[1,2]].set_index(1).to_dict(orient='index')
    boiling_dic = {k: v[2] for k, v in boiling_dic.items()}  # {'34': 1110.0, '35': 0.0, ...}
    total_kg = sum(boiling_dic.values())
    
    # round to get full
    # todo: proper logic
    total_kg = custom_round(total_kg, 850, rounding='floor')
    
    n_boilings = int(total_kg / 850)
    for i in range(n_boilings):
        cur_kg = 850
        boiling_request = {}
        for k, v in list(boiling_dic.items()):
            boil_kg = min(cur_kg, boiling_dic[k])
            
            boiling_dic[k] -= boil_kg
            cur_kg -= boil_kg
            
            if k not in boiling_request:
                boiling_request[k] = 0
            boiling_request[k] += boil_kg
            
            if cur_kg == 0:
                break
                
        if cur_kg != 0:
            # any non-zero
            print('Non-zero')
            k = [k for k, v in boiling_request.items() if v != 0][0]
            boiling_request[k] += cur_kg
        
        boiling_request = {k: v for k, v in boiling_request.items() if v != 0}
        values.append([boiling_id, boiling_request])
df = pd.DataFrame(values)
df[0] = df[0].astype(str)
df.head()

,0,1
0,1,{'34': 850}
1,1,"{'34': 260.0, '37': 590.0}"
2,1,"{'37': 60.39999999999998, '39': 789.6}"
3,1,{'39': 850.0}
4,1,{'39': 850.0}


In [30]:
skus.iloc[0][0].name

'Моцарелла "Pretto" (для бутербродов), 45%, 0,2 кг, т/ф, (9 шт)'

In [46]:
load_sources()
# draw salt and water
root = Block('root', t=cast_t('01:45'))

boilings = []

for i, row in df.iterrows():
    boiling_type = 'salt' if str(cast_boiling(row[0]).percent) == '2.7' else 'water'
#     if boiling_type == 'water':
#         continue
    b = make_boiling(cast_boiling(row[0]), {cast_sku(k): v for k, v in row[1].items()}, boiling_type, block_num=i + 1)
    iter_props = iter_water_props if b.props['boiling_type'] == 'water' else iter_salt_props
    dummy_push(root, b, iter_props=iter_props, validator=boiling_validator, beg='last_beg', max_tries=100)

In [47]:
root.rel_props['size'] = max(c.end for c in root.children)
draw_schedule(root, STYLE, 'output.xlsx', init_sheet)

termizator 5 (14,20] (21,27]
pouring_name 5 (20,41] (27,48]
pouring_and_fermenting 6 (14,26] (21,33]
soldification 6 (26,32] (33,39]
cutting 6 (32,36] (39,43]
pouring_off 6 (36,39] (43,46]
extra 6 (39,41] (46,48]
melting_label 14 (17,21] (96,100]
melting_name 14 (21,51] (100,130]
serving 15 (17,23] (96,102]
melting_process 15 (23,33] (102,112]
salting 16 (23,41] (102,120]
packing_label 26 (41.0,44.0] (120.0,123.0]
packing_name 26 (44.0,51.0] (123.0,130.0]
packing_brand 27 (41.0,51.0] (120.0,130.0]
termizator 7 (24,30] (31,37]
pouring_name 7 (30,51] (37,58]
pouring_and_fermenting 8 (24,36] (31,43]
soldification 8 (36,42] (43,49]
cutting 8 (42,46] (49,53]
pouring_off 8 (46,49] (53,56]
extra 8 (49,51] (56,58]
melting_label 17 (27,31] (106,110]
melting_name 17 (31,80] (110,159]
serving 18 (27,33] (106,112]
melting_process 18 (33,62] (112,141]
salting 19 (33,51] (112,130]
packing_label 26 (51.0,54.0] (130.0,133.0]
packing_name 26 (54.0,80.0] (133.0,159.0]
packing_brand 27 (51.0,80.0] (130.0

cutting 8 (287,291] (294,298]
pouring_off 8 (291,294] (298,301]
extra 8 (294,296] (301,303]
melting_label 20 (272,276] (351,355]
melting_name 20 (276,313] (355,392]
serving 21 (272,278] (351,357]
melting_process 21 (278,288] (357,367]
salting 22 (278,303] (357,382]
packing_label 26 (303.0,306.0] (382.0,385.0]
packing_name 26 (306.0,313.0] (385.0,392.0]
packing_brand 27 (303.0,313.0] (382.0,392.0]
termizator 5 (286,292] (293,299]
pouring_name 5 (292,313] (299,320]
pouring_and_fermenting 6 (286,298] (293,305]
soldification 6 (298,304] (305,311]
cutting 6 (304,308] (311,315]
pouring_off 6 (308,311] (315,318]
extra 6 (311,313] (318,320]
melting_label 14 (289,293] (368,372]
melting_name 14 (293,330] (372,409]
serving 15 (289,295] (368,374]
melting_process 15 (295,305] (374,384]
salting 16 (295,320] (374,399]
packing_label 26 (320.0,323.0] (399.0,402.0]
packing_name 26 (323.0,330.0] (402.0,409.0]
packing_brand 27 (320.0,330.0] (399.0,409.0]
termizator 1 (292,298] (299,305]
pouring_name 1 (29